In [16]:
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
make_upper = RunnableLambda(lambda x: x.upper())
make_lower = RunnableLambda(lambda x: x.lower())
chain =  make_upper | StrOutputParser()
result = chain.invoke("I am Gaja")
result

'I AM GAJA'

In [ ]:
def case_conversion(text):
    return text.upper()
make_upper = RunnableLambda(case_conversion)
chain =  make_upper | StrOutputParser()
result = chain.invoke("I am Gaja")
result

# RAG CHAIN

In [21]:
load_dotenv()
api = os.getenv('google_flash')

In [42]:
#Rag Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableMap, RunnableBranch
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=api,
    temperature=0.7,
    max_tokens=None,
)

template = '''
Consider yourself as a poet and write a poem for given topic
Topic : {topic}
'''

prompt = ChatPromptTemplate.from_template(template)
parser = StrOutputParser()

In [25]:
rag_chain = prompt | llm | parser
results = rag_chain.invoke({"topic":"Computers"})
results

"From silicon dreams, a new age unfurls,\nA whispered command that re-shapes our worlds.\nA titan born of logic, cold and keen,\nBehind a luminous, ever-shifting screen.\n\nIt hums with purpose, a relentless drive,\nWhere countless data points now swiftly thrive.\nA boundless library, a mind unbound,\nWhere every question quickly can be found.\n\nIt weaves a web, a vast, unseen design,\nConnecting souls across the digital line.\nFrom distant shores, a voice, a face, a glance,\nA global village, in a pixel dance.\n\nThe artist finds new canvas, bright and bold,\nThe bard composes sagas, to be told\nIn rhythmic code, or melodies that soar,\nUnlocking wonders never seen before.\n\nYet in its brilliance, shadows softly gleam,\nA digital illusion, a waking dream.\nFor human touch, a warmth it cannot hold,\nAnd stories left, in real-time, untold.\n\nWe feed it hours, our attention's toll,\nA silent hunger, claiming heart and soul.\nAnd sometimes wonder, in its sterile light,\nIf warmth is lo

In [39]:
# Super simple RAG that just returns relevant chunks
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Setup documents
knowledge_base = [
    "Python is a programming language created by Guido van Rossum.",
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning uses neural networks with multiple layers.",
    "LangChain helps build applications with LLMs.",
    "HuggingFace provides transformer models and datasets."
]

# 2. Create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 3. Create vector store
vectorstore = FAISS.from_texts(knowledge_base, embeddings)
def rag_reterive(query):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    docs = retriever.invoke(query)
    context = ("\n").join(doc.page_content for doc in docs)
    return context

In [43]:
prompt = ChatPromptTemplate.from_template("""
Use ONLY the context below to answer.
If context is insufficient, say "I don't know".

Context:
{context}

Question:
{question}
""")

rag_chain = (
    RunnableParallel({"context": RunnableLambda(rag_reterive),"question":RunnablePassthrough()})
    | prompt
    | llm
    | parser
)
question = "what is huggingface"
answer = rag_chain.invoke(question)
print(answer)

HuggingFace provides transformer models and datasets.


ModuleNotFoundError: No module named 'langchain.chains'